In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

seds = pd.read_csv('seds.csv') #1
revenue_to_provider = pd.read_csv('revenue_to_provider.csv') #2
filter_pack_concentrations_weekly = pd.read_csv('filter_pack_concentration_weekly.csv') #3
energy_workforce_training = pd.read_csv('energy_workforce_training.csv') #4
vehicles = pd.read_csv('vehicles.csv') #5
land_temp_by_city = pd.read_csv('land_temp_by_city.csv') #6
land_temp_by_state = pd.read_csv('land_temp_by_state.csv') #7
power_plants = pd.read_csv('power_plants.csv') #8

In [3]:
seds['energy_source'] = seds['msn'].apply(lambda v: v[:2])
seds['energy_sector'] = seds['msn'].apply(lambda v: v[2:4])
seds['energy_unit'] = seds['msn'].apply(lambda v: v[4])

In [27]:
import math

cities = dict()
for index, row in land_temp_by_city.iterrows():
    if row['city'] not in cities.keys():
        x = math.cos(row['latitude']) * math.cos(row['longitude'])
        y = math.cos(row['latitude']) * math.sin(row['longitude'])
        z = math.sin(row['latitude'])
        cities[row['city']] = (x,y,z)
        
temp_cities = pd.DataFrame.from_dict(cities, orient='index', columns=['x','y','z'])

cities = dict()
for index, row in power_plants.iterrows():
    if row['city'] not in cities.keys():
        x = math.cos(row['latitude']) * math.cos(row['longitude'])
        y = math.cos(row['latitude']) * math.sin(row['longitude'])
        z = math.sin(row['latitude'])
        cities[row['city']] = (x,y,z)
        
plants_cities = pd.DataFrame.from_dict(cities, orient='index', columns=['x','y','z'])

latitude                   39.9883
longitude                 -79.2516
state_code                      PA
county                    Somerset
date_on        01/03/2000 16:37:00
date_off       01/10/2000 16:03:00
TSO4                        2.3406
TNH4                        0.7764
CA                          0.1267
MG                          0.0144
NA                           0.174
K                           0.0536
CL                             NaN
NSO4                        0.4753
NHNO3                       1.1347
WSO2                       10.2084
WNO3                        0.5588
total_SO2                  10.5254
total_NO3                    1.893
flow_volume                14.9233
x                         0.498385
Name: 0, dtype: object

In [22]:
len(set(power_plants.city))
len(set(land_temp_by_city.city))

248

In [70]:
# add state_name column to dataset_6

power_plant_cities = dict()
for index, row in power_plants.iterrows():
    if (row['city']) not in power_plant_cities.keys():
        power_plant_cities[row['city']] = [(row['state_name'],row['latitude'], row['longitude'])]
    else:
        already_in = False
        for el in power_plant_cities[row['city']]:
            if el[0] == row['state_name']:
                already_in = True
        if not already_in:
            power_plant_cities[row['city']].append((row['state_name'],row['latitude'], row['longitude']))
            

def get_state(row, cities_dict):
    try:
        if len(cities_dict[row['city']])==1:
            return cities_dict[row['city']][0][0]
        else:
            return min(cities_dict[row['city']], lambda v: (v[0]-row['latitude'])**2 + (v[1]-row['longitude']**2))[0]
    except:
        return 'UNDEFINED'

land_temp_by_city['state_name'] = land_temp_by_city.apply(get_state,args=(power_plant_cities,), axis=1)


In [72]:
from math import cos, asin, sqrt

temp_cities_dict = dict()
for index, row in land_temp_by_city.iterrows():
    if (row['city'], row['state_name']) not in temp_cities_dict.keys():
        temp_cities_dict[(row['city'], row['state_name'])] = (row['latitude'], row['longitude'])
        
temp_cities = []
for key, item in cities.items():
    temp_cities.append({'city_state':key, 'latitude':item[0], 'longitude':item[1]})
    
power_cities_dict = dict()
for index, row in power_plants.iterrows():
    if (row['city'], row['state_name']) not in power_cities_dict.keys():
        power_cities_dict[(row['city'], row['state_name'])] = (row['latitude'], row['longitude'])
        
power_cities = []
for key, item in cities.items():
    power_cities.append({'city_state':key, 'latitude':item[0], 'longitude':item[1]})
        
        

In [78]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(v, data): # data = cities, v = row 
    x = min(data, key=lambda p: distance(v['latitude'],v['longitude'],p['latitude'],p['longitude']))
    return pd.Series([x['city_state'], distance(x['latitude'],x['longitude'],v['latitude'], v['longitude'])])

In [ ]:
filter_pack_concentrations_weekly[['temp_city', 'temp_city_dist']] = filter_pack_concentrations_weekly.apply(closest, args=(temp_cities,), axis=1)
filter_pack_concentrations_weekly[['power_city', 'power_city_dist']] = filter_pack_concentrations_weekly.apply(closest, args=(power_cities,), axis=1)




In [71]:
set(land_temp_by_city['state_name'])

{'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Florida',
 'Georgia',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kentucky',
 'Louisiana',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Missouri',
 'Nebraska',
 'Nevada',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Dakota',
 'Tennessee',
 'Texas',
 'UNDEFINED',
 'Utah',
 'Virginia',
 'Washington',
 'Wisconsin'}